In [ ]:
from google.colab import drive
import os
import sys

drive.mount('/content/drive')

PROJECT_PATH = '/content/drive/MyDrive/SVGEditor'
CODE_PATH = f"{PROJECT_PATH}/code"
MODEL_PATH = f"{PROJECT_PATH}/models"
RESOURCE_PATH = f"{PROJECT_PATH}/Resource"

if CODE_PATH not in sys.path:
    sys.path.insert(0, CODE_PATH)

# Single target processing
TARGET_NAME = "153_B"
ORIGINAL_PATH = f"{RESOURCE_PATH}/{TARGET_NAME}/Original"
INTERMEDIATE_FILES_PATH = f"{RESOURCE_PATH}/{TARGET_NAME}/IntermediateFiles"
COMPARISON_PATH = f"{RESOURCE_PATH}/{TARGET_NAME}/Comparison"
RESULT_PATH = f"{RESOURCE_PATH}/{TARGET_NAME}/Result"

# Input image file
INPUT_IMAGE = f"{ORIGINAL_PATH}/153_B.jpg"

# Vectorization input - the edited image from Qwen
VECTORIZATION_INPUT = f"{INTERMEDIATE_FILES_PATH}/153_B_qwen_edit.png"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Install dependencies (removed duplicates)
%pip install diffusers accelerate safetensors transformers huggingface_hub segment-anything opencv-python pillow matplotlib scikit-image scikit-learn cairosvg moviepy shapely networkx lxml

# Install vectorization dependencies
%pip install skan rdp

import shutil
import subprocess

# 1. Clean existing code directory and clone fresh repository
if os.path.exists(CODE_PATH):
    shutil.rmtree(CODE_PATH)

os.chdir(PROJECT_PATH)
result = subprocess.run(['git', 'clone', 'https://github.com/huanbasara/SVGEditor.git', 'code'],
                       capture_output=True, text=True)
print(f"Code repository {'successfully' if result.returncode == 0 else 'failed'} cloned")

# 2. Display latest commit information
os.chdir(CODE_PATH)
commit_info = subprocess.run(['git', 'log', '-1', '--pretty=format:%h|%ci|%s'],
                           capture_output=True, text=True)

if commit_info.returncode == 0:
    hash_code, commit_time, commit_msg = commit_info.stdout.strip().split('|', 2)
    print(f"Latest commit: {hash_code} ({commit_time.split()[0]}) - {commit_msg}")

# 3. Add code path to Python sys.path
if CODE_PATH not in sys.path:
    sys.path.insert(0, CODE_PATH)
    print(f"Added {CODE_PATH} to Python path")

# 4. Clear custom modules from cache
modules_to_clear = ['sam_processor', 'svglib', 'utils']

for base in modules_to_clear:
    to_remove = [m for m in sys.modules if m.startswith(base)]
    for m in to_remove:
        del sys.modules[m]

print("Modules reloaded!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.0/128.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.6/100.6 kB 6.8 MB/s eta 0:00:00
  Created wheel for rdp: filename=rdp-0.8-py3-none-any.whl size=4585 sha256=26a5d562c2d183f2df38f94477f071d27f7faa71399d6804fc990ff3ad62fa82
  Stored in directory: /root/.cache/pip/wheels/3e/80/75/e2f0e8372fde60e3be87cc64b088e10b6ed78589719d69653e
Successfully built rdp
Code repository successfully cloned
Latest commit: 39b6297 (2025-10-01) - update@2025-10-01 10:48:20
Modules reloaded!


In [ ]:
# ==================== 骨架到SVG转换 ====================

import os
import cv2
import numpy as np
from skimage.measure import label, regionprops
from skimage.morphology import remove_small_objects
from svglib.svg import SVG
from svglib.svg_path import SVGPath
from svglib.svg_primitive import SVGPathGroup
from svglib.geom import Bbox
import matplotlib.pyplot as plt

def extract_open_paths(skeleton):
    """从骨架提取开放路径"""
    skeleton_binary = (skeleton > 0).astype(np.uint8)
    labeled = label(skeleton_binary)
    regions = regionprops(labeled)

    paths = []
    for region in regions:
        coords = region.coords
        if len(coords) < 2:
            continue

        # 计算每个点的邻居数量
        def count_neighbors(point, all_coords):
            cnt = 0
            for c in all_coords:
                if (abs(c[0] - point[0]) <= 1 and abs(c[1] - point[1]) <= 1 and tuple(c) != tuple(point)):
                    cnt += 1
            return cnt

        endpoints = [p for p in coords if count_neighbors(p, coords) == 1]
        remaining = set(tuple(c) for c in coords)

        while remaining:
            # 优先从端点开始
            start = None
            if endpoints:
                for ep in endpoints:
                    if tuple(ep) in remaining:
                        start = ep
                        break
            if start is None:
                start = next(iter(remaining))

            # 追踪路径
            path_coords = []
            cur = start
            path_coords.append((int(cur[1]), int(cur[0])))
            remaining.remove(tuple(cur))

            while remaining:
                nxt = None
                for c in list(remaining):
                    if (abs(c[0] - cur[0]) <= 1 and abs(c[1] - cur[1]) <= 1):
                        nxt = c
                        break
                if nxt is None:
                    break
                path_coords.append((int(nxt[1]), int(nxt[0])))
                remaining.remove(nxt)
                cur = nxt

            if len(path_coords) >= 2:
                paths.append(path_coords)

    return paths

def build_svg_paths(paths):
    """将路径转换为SVG路径对象"""
    svg_paths = []
    for path in paths:
        d = f"M {path[0][0]:.2f} {path[0][1]:.2f}"
        for x, y in path[1:]:
            d += f" L {x:.2f} {y:.2f}"
        path_group = SVGPath.from_str(d, add_closing=False)
        svg_paths.extend(path_group.svg_paths)
    return svg_paths

def visualize_results(skeleton, paths):
    """可视化对比结果"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 7))

    # 原始骨架
    axes[0].imshow(skeleton, cmap='gray')
    axes[0].set_title('Original Skeleton')
    axes[0].axis('off')

    # 骨架 vs 提取的路径对比图
    axes[1].imshow(skeleton, cmap='gray', alpha=0.3)
    for path in paths:
        path_array = np.array(path)
        axes[1].plot(path_array[:, 0], path_array[:, 1], 'r-', linewidth=0.8, alpha=0.8)
    axes[1].set_title(f'Skeleton vs Extracted Paths ({len(paths)} paths)')
    axes[1].axis('off')

    plt.tight_layout()
    plt.show()

# 主流程
skeleton_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_skeleton.png")
if os.path.exists(skeleton_path):
    skeleton = cv2.imread(skeleton_path, cv2.IMREAD_GRAYSCALE) > 0
    paths = extract_open_paths(skeleton)

    if paths:
        svg_paths = build_svg_paths(paths)
        viewbox = Bbox(0, 0, skeleton.shape[1], skeleton.shape[0])
        group = SVGPathGroup(svg_paths, fill=False, stroke_width=2.0)

        # 确保路径有filling属性
        for sp in group.svg_paths:
            if not hasattr(sp, 'filling'):
                sp.filling = 0

        svg = SVG([group], viewbox=viewbox)
        raw_svg_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_raw.svg")
        svg.save_svg(raw_svg_path, coordinate_precision=1)

        print(f"✅ 提取了 {len(paths)} 条路径，已保存到: {raw_svg_path}")

        # 可视化结果
        visualize_results(skeleton, paths)
    else:
        print("❌ 路径提取失败")
else:
    print("❌ 骨架图片不存在")